In [44]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

In [45]:
# Connexion à MariaDB
def create_connection(host_name, user_name, user_password, db_name=None, port=3306):
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name,
            port=port,
            connection_timeout=600  # 10 minutes de délai
        )
        print(f"Connexion réussie à la base de données {db_name}" if db_name else "Connexion réussie au serveur MariaDB")
        return connection
    except Error as e:
        print(f"Erreur : '{e}'")
        return None



In [46]:
def creer_table_si_absente_old(cursor, nom_table, colonnes,dtype):
    """
    Crée une table dans la base de données si elle n'existe pas déjà,
    en utilisant les colonnes extraites de l'en-tête du fichier CSV.
    """
    colonnes_avec_types = []
    for colonne in colonnes:
        type_colonne = dtype.get(colonne, 'TEXT')  # Utiliser 'TEXT' par défaut si le type n'est pas spécifié
        if type_colonne == 'float':
            type_colonne = 'REAL'
        elif type_colonne == 'int':
            type_colonne = 'INTEGER'
        colonnes_avec_types.append(f"{colonne} {type_colonne}")
    colonnes_avec_types_str = ', '.join(colonnes_avec_types)
    requete_creation = f"CREATE TABLE IF NOT EXISTS {nom_table} ({colonnes_avec_types_str});"
    print(requete_creation)
    cursor.execute(requete_creation)


In [47]:
def creer_table_si_absente(connection, nom_table, colonnes, dtype):
    """
    Crée une table dans MariaDB si elle n'existe pas déjà,
    en utilisant les colonnes extraites et leurs types.
    """
    colonnes_avec_types = []
    
    for colonne in colonnes:
        # On récupère le type depuis dtype ou on met 'VARCHAR(255)' par défaut
        type_colonne = dtype.get(colonne, 'VARCHAR(255)')
        
        # Ajustement des types pour MariaDB
        if type_colonne == 'float':
            type_colonne = 'FLOAT'
        elif type_colonne == 'int':
            type_colonne = 'INT'
        elif type_colonne == 'str':
            type_colonne = 'VARCHAR(255)'  # 'VARCHAR' est généralement utilisé pour les strings
        
        colonnes_avec_types.append(f"{colonne} {type_colonne}")
    
    # Génère la requête SQL avec les colonnes et leurs types
    colonnes_avec_types_str = ', '.join(colonnes_avec_types)
    
    # Requête pour créer la table si elle n'existe pas déjà
    requete_creation = f"CREATE TABLE IF NOT EXISTS {nom_table} ({colonnes_avec_types_str});"
    
    print(requete_creation)  # Pour déboguer, afficher la requête générée
    
    cursor=connection.cursor()
    try:
        cursor.execute(requete_creation)  # Exécuter la requête
        #connection.commit()
        print(f"Table '{nom_table}' créée avec succès (si absente)")
    except Error as e:
        print(f"Erreur lors de la création de la table : '{e}'")


In [48]:
def inserer_donnees_old(cursor, nom_table, colonnes, donnees):
    """
    Insère les données dans la table. 
    """
    placeholders = ', '.join(['?' for _ in colonnes])
    colonnes_str = ', '.join(colonnes)
    requete_insertion = f"INSERT INTO {nom_table} ({colonnes_str}) VALUES ({placeholders});"
    cursor.executemany(requete_insertion, donnees)

In [49]:
def inserer_donnees(connection, nom_table, colonnes, donnees):
    """
    Insère les données dans la table MariaDB.
    """
    # Création des placeholders pour MariaDB (%s)
    placeholders = ', '.join(['%s' for _ in colonnes])
    # Génère une chaîne avec les noms de colonnes
    colonnes_str = ', '.join(colonnes)
    # Création de la requête d'insertion
    requete_insertion = f"INSERT INTO {nom_table} ({colonnes_str}) VALUES ({placeholders});"
    cursor=connection.cursor()
    try:
        # Exécution de l'insertion avec executemany
        cursor.executemany(requete_insertion, donnees)
        # Commit pour enregistrer les changements dans la base de données
        #connection.commit()
        print(f"{cursor.rowcount} lignes insérées avec succès dans la table {nom_table}")
    except Error as e:
        print(f"Erreur lors de l'insertion : '{e}'")


In [50]:
def extraire_info_du_nom_fichier(fichier,liste_nom_table):  
    # extraction informations du nom de fichier à encoder dans la table
    annee_mois=fichier[:7]
    info_geo=fichier[8:-4]
    # where sans l'élément qui est dans la liste
    for i in liste_nom_table:
        if i in info_geo:
            info_geo=info_geo.replace(i,"")[:-1]
    return annee_mois,info_geo

In [51]:
def nettoyer_noms_colonnes(colonnes):
    """
    Nettoie les noms des colonnes pour éviter les problèmes de requêtes SQL.
    """
     # nettoyage des noms de colonnes
    colonnes = [colonne.replace(' ', '') for colonne in colonnes]
    colonnes = [colonne.replace('-', '') for colonne in colonnes]
    colonnes = [colonne.replace('_', '') for colonne in colonnes]
    colonnes_nettoyees = [colonne.replace(' ', '').replace('-', '').replace('_', '') for colonne in colonnes]
    return colonnes_nettoyees

In [52]:
def nettoyer_nom_table(liste_nom_table,chemin_fichier):
    for nom_table in liste_nom_table:
        if nom_table in chemin_fichier:
            nom_table = nom_table.replace('-', '')
            break
        else:
            nom_table = 'autre'
            
    print(f"nom de la table : {nom_table}")
    # Créer la table si elle n'existe pas
    return nom_table


In [53]:
def nettoyer_donnees(donnees):
    """
    Remplace les NaN dans les données par des valeurs par défaut.
    Pour les chaînes, remplace par une chaîne vide, pour les numériques, par NULL.
    """
    # Remplace les NaN par une chaîne vide pour les colonnes de type chaîne
    donnees = donnees.replace({np.nan: ''})
    
    # Si besoin, remplacez les NaN par NULL dans les colonnes numériques
    # (laisser en blanc ici signifie que MariaDB l'insérera comme NULL)
    
    return donnees


In [54]:
def traiter_fichier(connection,chemin_fichier, fichier,liste_nom_table,dtype):
    """
    Traite un fichier CSV en créant une table correspondante et en y insérant les données.
    """
    # Gérer les valeurs manquantes
    na_values = ['NA', 'N/A', '']

    df = pd.read_csv(chemin_fichier, dtype=dtype, na_values=na_values)

    # rajout des informations year_month et where dans le dataframe
    annee_mois,info_geo = extraire_info_du_nom_fichier(fichier,liste_nom_table)
    df['annee_mois'] = annee_mois
    df['info_geo'] = info_geo
    
    # nettoyer les données
    df = nettoyer_donnees(df)

    nom_table = nettoyer_nom_table(liste_nom_table,chemin_fichier)
    colonnes = nettoyer_noms_colonnes(list(df.columns))

    # Insérer les données dans la table
    
    creer_table_si_absente(connection,nom_table, colonnes,dtype)
    inserer_donnees(connection,nom_table, colonnes, df.values.tolist())

    print(f"Traitement du fichier : {chemin_fichier} terminé.")

    return df

In [55]:
def parcourir_arborescence(connection,chemin_racine, db_path,liste_nom_table,dtype):
    """
    Parcourt récursivement l'arborescence et traite chaque fichier CSV trouvé.
    """
    
    for racine, sous_repertoires, fichiers in os.walk(chemin_racine):
         for fichier in fichiers:
            if fichier.endswith(".csv"):
                chemin_fichier = os.path.join(racine, fichier)
                df=traiter_fichier(connection,chemin_fichier, fichier,liste_nom_table,dtype)
    return df

In [57]:

chemin_racine = "D:/ICAM/Crimes"

db_name = "crime" 

# on définit le nom des tables en fonction du nom du fichier (terminaison)
liste_nom_table = ['outcomes','stop-and-search','street']

dtype={'Longitude': 'float', 'Latitude': 'float'}

# Connexion à la base de données 'crime'
connection = create_connection("127.0.0.1", "root", "", db_name)

if connection:
    df=parcourir_arborescence(connection,chemin_racine, db_name,liste_nom_table,dtype)
    # Fermeture de la connexion
    if connection.is_connected():
        connection.commit()
        connection.close()
        print("Connexion MariaDB fermée")
  

Connexion réussie à la base de données crime
nom de la table : outcomes
CREATE TABLE IF NOT EXISTS outcomes (CrimeID VARCHAR(255), Month VARCHAR(255), Reportedby VARCHAR(255), Fallswithin VARCHAR(255), Longitude FLOAT, Latitude FLOAT, Location VARCHAR(255), LSOAcode VARCHAR(255), LSOAname VARCHAR(255), Outcometype VARCHAR(255), anneemois VARCHAR(255), infogeo VARCHAR(255));
Table 'outcomes' créée avec succès (si absente)
2527 lignes insérées avec succès dans la table outcomes
Traitement du fichier : D:/ICAM/Crimes\2019-11\2019-11-avon-and-somerset-outcomes.csv terminé.
nom de la table : stopandsearch
CREATE TABLE IF NOT EXISTS stopandsearch (Type VARCHAR(255), Date VARCHAR(255), Partofapolicingoperation VARCHAR(255), Policingoperation VARCHAR(255), Latitude FLOAT, Longitude FLOAT, Gender VARCHAR(255), Agerange VARCHAR(255), Selfdefinedethnicity VARCHAR(255), Officerdefinedethnicity VARCHAR(255), Legislation VARCHAR(255), Objectofsearch VARCHAR(255), Outcome VARCHAR(255), Outcomelinkedt

KeyboardInterrupt: 

In [86]:
df.head(2)

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,annee_mois,info_geo
0,NaN,2021-10,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.494870,51.422276,On or near Conference/Exhibition Centre,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN,2021-10,avon-and-somerset
1,NaN,2021-10,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.511761,51.409966,On or near Caernarvon Close,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN,2021-10,avon-and-somerset


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14554 entries, 0 to 14553
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Crime ID               12241 non-null  object 
 1   Month                  14554 non-null  object 
 2   Reported by            14554 non-null  object 
 3   Falls within           14554 non-null  object 
 4   Longitude              13411 non-null  float64
 5   Latitude               13411 non-null  float64
 6   Location               14554 non-null  object 
 7   LSOA code              13411 non-null  object 
 8   LSOA name              13411 non-null  object 
 9   Crime type             14554 non-null  object 
 10  Last outcome category  12241 non-null  object 
 11  Context                0 non-null      float64
 12  annee_mois             14554 non-null  object 
 13  info_geo               14554 non-null  object 
dtypes: float64(3), object(11)
memory usage: 1.6+ MB


In [67]:
# exécuter la requête  suivante 
requete = """
select * from outcomes as o join street as s on o.`CrimeID` = s.`CrimeID` 
where o.`Reportedby` != s.`Reportedby`  or o.`Fallswithin` != s.`Fallswithin`
or o.Longitude != s.Longitude or s.Latitude != o.Latitude 
or o.Location != s.Location or o.`LSOAcode` != s.`LSOAcode` 
or o.`LSOAname` != s.`LSOAname`
"""

connection = sqlite3.connect(db_path)
df = pd.read_sql_query(requete, connection)
print(df)
connection.close()
